[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/yue-sun/generative-art/blob/main/02_tuesday/01_voronoi_delaunay_art.ipynb)

# Voronoi-Delaunay Image Art 
Jiayin Lu, Dec 2022, Jan 2023 mini-course

![Voro_art_preview.PNG](https://raw.githubusercontent.com/yue-sun/generative-art/master/02_tuesday/figs/voro_art_preview.PNG)

## Learning goals:
 - Voronoi diagram
 - Delaunay triangulation
 - Geometry meshing
 - Image color representation (and manipulation)

## Artistic goal: 
 - Photo art using the above!

## Overview
    1. Voronoi Diagram
        1.1. A simple demo with random points
        1.2. Photo art time!
            1.2.1. Color representation
            1.2.2. Generate Voronoi diagram overlaying the image
            1.2.3. Use input points' RGB to fill their Voronoi cells' colors
    2. Delaunay triangulation
        2.1. Simple demo with random points
        2.2. Photo art time!
            2.2.1. Bilinear interpolation for coloring 
            2.2.2. Use triangle vertex RGB and bilinear interpolation to fill the triangle cells colors
    3. Geometry meshing
        3.1. Signed Distance Function (SDF): shape representation
        3.2. Centroidal Voronoi Diagram (CVD) and Lloyd's Algorithm
            3.2.1. Centroidal Voronoi Diagram
            3.2.2. Lloyd's algorithm
        3.3. Meshing algorithm
            3.3.1. Trim points inside shape
            3.3.2. Project outside points back to geometry boundary
            3.3.3. Lloyd's iterations and projection of outside points
            3.3.4. Delaunay triangulation and coloring
    4. Free creating! Be creative :)

In [ ]:
#Import relevant libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.spatial import Delaunay
import matplotlib.tri as mtri
import matplotlib.colors as clr
import requests
from io import BytesIO
from PIL import Image

# 1. Voronoi diagram

![Voro_concept.PNG](https://raw.githubusercontent.com/yue-sun/generative-art/master/02_tuesday/figs/Voro_concept.PNG)

![Voro_app_1.PNG](https://raw.githubusercontent.com/yue-sun/generative-art/master/02_tuesday/figs/Voro_app_1.PNG)

![Voro_app_2.png](https://raw.githubusercontent.com/yue-sun/generative-art/master/02_tuesday/figs/Voro_app_2.PNG)

## 1.1. A simple demo with random points 

Set random seed, generate random points, and compute the Voronoi diagram of the points.

In [ ]:
#https://numpy.org/doc/stable/reference/random/generator.html
seed=10
rng = np.random.default_rng(seed) #random number generator
points = rng.integers(low=0,high=100,size=(10,2),endpoint=True) 
#print(points)

In [ ]:
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.Voronoi.html
vor = Voronoi(points)

In [ ]:
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.voronoi_plot_2d.html
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
voronoi_plot_2d(vor,ax)
plt.show()

Get different information of the Voronoi cells.

In [ ]:
#Input points/generators
vor.points

In [ ]:
#Voronoi cell vertices
vor.vertices

In [ ]:
#Input points neighbor pairs (ID): 
#Between each pair there lies a Voronoi cell boundary
vor.ridge_points

In [ ]:
#Voronoi vertices (ID) forming each Voronoi edge
vor.ridge_vertices

In [ ]:
#Indices of the Voronoi vertices forming each Voronoi region. 
#-1 indicates vertex outside the Voronoi diagram.
vor.regions

In [ ]:
#Index of the Voronoi region for each input point.
#For example, here, point 0 is associated with the 2nd Voronoi region, [2,-1]
vor.point_region

As we can see, it is very useful to assign ID to points, and refer to points with their IDs. 

The IDs are also very useful in linking related information.

It is also very common to use dummy values, like "-1", to represent special case values, that we may ignore sometimes. 

We can use the above information as needed later.

## 1.2. Photo art time!

Import an image and view

In [ ]:
url = 'https://raw.githubusercontent.com/yue-sun/generative-art/main/02_tuesday/puppy.jpg'
page = requests.get(url)
img_file=Image.open(BytesIO(page.content))
img = np.asarray(img_file)
img=np.flip(img,axis=0)

In [ ]:
dim_x=len(img[0,:])
dim_y=len(img[:,0])
print("dim_x, {}; dim_y, {}".format(dim_x,dim_y))

In [ ]:
plt.figure(figsize=(10,int(10*dim_y/dim_x)))
plt.imshow(img,origin="lower")

## 1.2.1. Color representation

Let's look at the img array to understand how color at each pixel point is represented.

In [ ]:
print(img[0,0:3]) #first row, first three pixels

The color at each pixel is represented by three interger numbers, in the R(ed), G(reen), B(lue) channels, in the range of [0,255].
You can look up colors here too: https://www.rapidtables.com/web/color/RGB_Color.html

In python plotting, we can also specify the color of plotting, by converting the color from [0,255] to floating number [0,1]. You can look up colors here: https://www.tug.org/pracjourn/2007-4/walden/color.pdf

For example, if we want to plot points with the color [106,162,221], we convert it by dividing all numbers by 255. And use a "tuple" of float values (r,g,b) to represent the colors.

In [ ]:
r=106/255
g=162/255
b=221/255
plt.plot(points[:,0],points[:,1],"*",color=(r,g,b))

## 1.2.2. Generate Voronoi diagram overlaying the image

First, let's generate a new set of points, that covers the image region. I use a "eps" here to generate points in a range slightly larger than the image dimensions range.

In [ ]:
points_img = [] 
eps=50
Np=5000
for i in range(Np):
    points_img.append([rng.integers(low=-eps,high=dim_x+eps,endpoint=True), \
                   rng.integers(low=-eps,high=dim_y+eps,endpoint=True)])
points_img = np.array(points_img)

In [ ]:
vor_img = Voronoi(points_img)

View the Voronoi diagram overlaying the image.

In [ ]:
fig = plt.figure(figsize=(20,int(20*dim_y/dim_x)))
ax = fig.add_subplot(111)
ax.imshow(img)
voronoi_plot_2d(vor_img,ax=ax)
plt.show()

## 1.2.3. Use input points' RGB to fill their Voronoi cells' colors

Now, we may create a mosaic version of the image by simply reading in the RGB value of the image at each input/generator point of the Voronoi diagram, and assign to the Voronoi cells the associated colors. 

In [ ]:
# plot Voronoi diagram, and fill finite regions with color mapped from speed value

# set figure size
fig = plt.figure(figsize=(20,int(20*dim_y/dim_x)))
ax = fig.add_subplot(111)

# plot Voronoi diagram
voronoi_plot_2d(vor_img, ax=ax,show_points=False, show_vertices=False, line_width=0.1, line_alpha=0.5)

# for each input/generator point, fill color of its Voronoi cell
for pi in range(0,Np):
    
    # generator point coordinate [px,py]
    px=points_img[pi,0]
    py=points_img[pi,1]
    
    # if the point is inside the image range
    if px>=0 and px<dim_x and py>=0 and py<dim_y:
        
        #get the corresponding RGB value, convert to float tuple
        rgb_img= tuple(img[py,px]/255)
        
        #get the Voronoi region associated with the input/generator point
        ri=vor_img.point_region[pi]
        
        #if the Voronoi region is valid and finite, 
        #fill the polygon defined by the Voronoi cell with the color
        if ri!=-1:
            region = vor_img.regions[ri]
            if not -1 in region:
                polygon = [vor_img.vertices[i] for i in region]
                ax.fill(*zip(*polygon), color=rgb_img)

#hide plot axis
ax.axis('off')
#set plotting x range and y range
plt.xlim((0,dim_x))
plt.ylim((0,dim_y))
plt.show()

# 2. Delaunay triangulation

![Delaunay_concept.PNG](https://raw.githubusercontent.com/yue-sun/generative-art/master/02_tuesday/figs/Delaunay_concept.PNG)

![Delaunay_app_1.PNG](https://raw.githubusercontent.com/yue-sun/generative-art/master/02_tuesday/figs/Delaunay_app_1.PNG)

## 2.1. Simple demo with random points

Overlay Delaunay triangulation with Voronoi diagram:

Delaunay triangulation: https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.Delaunay.html

To draw the triangulation, we can use plt.triplot: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.triplot.html

In [ ]:
seed=10
rng = np.random.default_rng(seed) #random number generator
points = rng.integers(low=0,high=100,size=(10,2),endpoint=True) 

In [ ]:
vor = Voronoi(points)
tri = Delaunay(points)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)

#plot the Voronoi diagram
voronoi_plot_2d(vor,ax)

#plot the Delaunay triangulation
plt.triplot(points[:,0], points[:,1], tri.simplices, "-o")

plt.show()


Get different information of the triangle elements:

In [ ]:
#Point IDs forming each triangle
tri.simplices

In [ ]:
#Point coordinates forming each triangle
points[tri.simplices]

## 2.2. Photo art time!

Similar to before, first, we overlay Delaunay triangulation on the image. 

In [ ]:
#generate random points based on the image dimensions
points_img = [] 
eps=50
Np=5000
for i in range(Np):
    points_img.append([rng.integers(low=-eps,high=dim_x+eps,endpoint=True), \
                   rng.integers(low=-eps,high=dim_y+eps,endpoint=True)])
points_img = np.array(points_img)

In [ ]:
#Delaunay triangulation
tria_img = Delaunay(points_img)

In [ ]:
#View the triangulation overlaying the image
fig = plt.figure(figsize=(20,int(20*dim_y/dim_x)))
ax = fig.add_subplot(111)
ax.imshow(img,origin="lower")
plt.triplot(points_img[:,0], points_img[:,1], tria_img.simplices,"-o")
plt.show()

## 2.2.1. Bilinear interpolation for coloring 

We can try a different coloring scheme for each triangle cells. 

For each triangle, we can read three RGB values at its three vertices. Then, we may use bilinear triangulation to fill the color inside the triangle cell. Each vertex of the triangle will have the exact RGB color they are assigned. For the area in between, the color is defined linearly, by the linear plane defined by the three points' values.

The coloring we get from this scheme is continuous, but not differentiable. 

![bilinear_tria.PNG](https://raw.githubusercontent.com/yue-sun/generative-art/master/02_tuesday/figs/bilinear_tria.PNG)

Recommended reading: https://en.wikipedia.org/wiki/Bilinear_interpolation

Here, we can simply use the functions provided in Python to do the plotting with bilinear interpolation of colors.

## 2.2.2. Use triangle vertex RGB and bilinear interpolation to fill the triangle cells colors

First, let's trim the number of triangles, and keep only triangles with all three vertices inside the image. That is, keep only triangles with color values defined on all three vertices. 

In [ ]:
#original triangle count
tria_ct=len(tria_img.simplices)
tria_ct

In [ ]:
#trimmed triangles 
tria_simplices_trim=[]

#loop through each original triangle
for ti in range(0,tria_ct):
    
    #get the three vertex coordinates
    v1id=tria_img.simplices[ti][0]
    v2id=tria_img.simplices[ti][1]
    v3id=tria_img.simplices[ti][2]
    v1x=points_img[v1id,0]
    v2x=points_img[v2id,0]
    v3x=points_img[v3id,0]
    v1y=points_img[v1id,1]
    v2y=points_img[v2id,1]
    v3y=points_img[v3id,1]
    
    #check all three vertices' coordinates are inside image
    if v1x>=0 and v1x<dim_x and v1y>=0 and v1y<dim_y:
        if v2x>=0 and v2x<dim_x and v2y>=0 and v2y<dim_y:
            if v3x>=0 and v3x<dim_x and v3y>=0 and v3y<dim_y:
                
                #store the valid triangle
                tria_simplices_trim.append(tria_img.simplices[ti])

tria_simplices_trim=np.array(tria_simplices_trim)

In [ ]:
#trimmed triangle count
tria_ct_trim=len(tria_simplices_trim)
tria_ct_trim

Next, we can loop through each point, and obtain the point's associated RGB value from the image, if the point lies inside the iamge:

In [ ]:
ver_rgb = np.zeros((Np,3))
for pi in range(0,Np):
    # generator point coordinate [px,py]
    px=points_img[pi,0]
    py=points_img[pi,1]
    
    # if the point is inside the image range
    if px>=0 and px<dim_x and py>=0 and py<dim_y:
        
        #get the corresponding RGB value, convert to float tuple
        ver_rgb[pi,:]= img[py,px]/255

In [ ]:
#take a look at the RGB associated with each point: 
ver_rgb

To use bilinear interpolation for the triangle element coloring, we can use plt.tripcolor(...) with the option "shading="gouraud"": https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.tripcolor.html

There is a complication here: the function does not allow explicit input of RGB color. But we can work around the issue by defining a colormap to pass in ourselves. The colormap basically creates a linear segmented look up table for each vertex's RGB color.

In [ ]:
#create our own color map for use in tripcolor(...) plotting later
z = np.arange(len(ver_rgb))
cmap = clr.LinearSegmentedColormap.from_list(
    "mymap", ver_rgb, N=len(ver_rgb)
)

In [ ]:
#create a triangulation object for use in tripcolor(...) plotting later
triang_obj = mtri.Triangulation(points_img[:,0], points_img[:,1], tria_simplices_trim)

In [ ]:
# plot Delaunay triangulation, and fill triangles inside the image with bilinear color interpolation

# set figure size
fig = plt.figure(figsize=(20,int(20*dim_y/dim_x)))
ax = fig.add_subplot(111)

# plot Delaunay triangulation
ax.triplot(points_img[:,0], points_img[:,1], tria_simplices_trim,linewidth=0.1,alpha=0.5,color="black")

#bilinear interpolation coloring
ax.tripcolor(triang_obj, z, shading="gouraud", cmap=cmap)

#hide plot axis
ax.axis('off')
#set plotting x range and y range
plt.xlim((0,dim_x))
plt.ylim((0,dim_y))
plt.show()

# 3. Geometry meshing

We can try some simple geometry meshing. As a simple example, maybe we can try:
- doing the delaunay triangulation restricted by a circular shape;
- AND the triangle mesh should be "nicely spaced" and "nicely shaped", rather than "randomly distributed".

## 3.1. Signed Distance Function (SDF): shape representation

![sdf_illustration.PNG](https://raw.githubusercontent.com/yue-sun/generative-art/master/02_tuesday/figs/sdf_illustration.PNG)

In [ ]:
#signed distance function for point (x,y) of a circle centered at (x0,y0) with radius r.
def sdf_circle(x,y,x0,y0,r):
    return np.sqrt((x-x0)**2+(y-y0)**2)-r

In [ ]:
#create arrays of X and Y: 50 evenly spaced values in between 0 and 2
X=np.linspace(0,2,50)
Y=np.linspace(0,2,50)
#create a 2D array to store the SDF value correspond to each (x,y) value
sdf_array=np.zeros((50,50))
#compute the SDF array values
for i in range(0,50):
    for j in range(0,50):
        x=X[i]
        y=Y[j]
        #SDF of (x,y) for a circle centered at (1,1) with radius 0.5
        sdf_array[j,i]=sdf_circle(x,y,1,1,0.5)

In [ ]:
#visualize the SDF field
plt.figure(figsize=(10,8))
plt.pcolor(X,Y,sdf_array,cmap="seismic")
plt.clim(-0.8,0.8)
plt.colorbar()

## 3.2. Centroidal Voronoi Diagram (CVD) and Lloyd's Algorithm

## 3.2.1. Centroidal Voronoi Diagram

Reference: Du, Q., Faber, V., and Gunzburger, M. 1999. Centroidal Voronoi tessellations: Applications and algorithms. SIAM Review 41, 4, 637--676.

https://people.sc.fsu.edu/~jburkardt/classes/urop_2016/du_faber_gunzburger.pdf

For simplicity purpose, we assume a uniform density function field in what follows.

![CVD_concept.PNG](https://raw.githubusercontent.com/yue-sun/generative-art/master/02_tuesday/figs/CVD_concept.PNG)

![CVD_app_2.PNG](https://raw.githubusercontent.com/yue-sun/generative-art/master/02_tuesday/figs/CVD_app_2.PNG)

![CVD_app_1.PNG](https://raw.githubusercontent.com/yue-sun/generative-art/master/02_tuesday/figs/CVD_app_1.PNG)

## 3.2.2. Lloyd's algorithm

![lloyds_alg.PNG](https://raw.githubusercontent.com/yue-sun/generative-art/master/02_tuesday/figs/lloyds_alg.PNG)

To implement the above algorithm, we first need to create a bounded Voronoi diagram, where the Voronoi diagram has a rectangular bound (ax,bx,ay,by). This can be achieved by adding "mirrored points" of the original boundary points in the four directions (up, down, left, right). Then, the original points' Voronoi diagram will be bounded by the rectangular box.

#### The following code taken from the previous MIT version of the course, https://gvarnavides.com/generative-art-workshop-website/docs/01.25-Tuesday/space-filling-curves. Nice code!!! XD

In [ ]:

# Compute a Voronoi diagram with rectangular bounds.
def bounded_voronoi(pts, bounds):
    # create the initial Voronoi diagram.
    vor = Voronoi(pts, incremental=True)

    # find vertices that lie outside the bounded domain, and store their index.
    # Also include -1, which is scipy's flag for a vertex outside the Voronoi diagram.
    out_vert_ids = [-1]+[i for i,v in enumerate(vor.vertices) if not in_bounds(v, bounds)]

    # identify which points have cell vertices outside the domain, and store their index.
    bound_ids = [i for i,r in enumerate(vor.point_region)
                 if any(v in out_vert_ids for v in vor.regions[r])]

    # mirror the boundary points and add them to the diagram. This makes the cells of
    # all our original points bounded in the rectangular domain.
    pts_add = mirror_points(pts[bound_ids,:], bounds)
    vor.add_points(pts_add)
    return vor

def mirror_points(pts, bounds):
    # inspired by: https://stackoverflow.com/questions/28665491/getting-a-bounded-polygon
    #-coordinates-from-voronoi-cells.
    # mirror the points over the boundaries of the bounding box to obtain bounded
    # polygons for all voronoi cells of interest.
    xmin, xmax, ymin, ymax = bounds
    xmid = 0.5*(xmin+xmax)
    ymid = 0.5*(ymin+ymax)
    pts_n = np.copy(pts[pts[:,1]>ymid]); pts_s = np.copy(pts[pts[:,1]<ymid])
    pts_e = np.copy(pts[pts[:,0]>xmid]); pts_w = np.copy(pts[pts[:,0]<xmid])
    pts_n[:,1] = ymax+(ymax-pts_n[:,1])
    pts_s[:,1] = ymin-(pts_s[:,1]-ymin)
    pts_e[:,0] = xmax+(xmax-pts_e[:,0])
    pts_w[:,0] = xmin-(pts_w[:,0]-xmin)
    pts_c = np.vstack([pts_n,pts_s,pts_e,pts_w])
    return pts_c

def in_bounds(pt, bounds):
    xmin, xmax, ymin, ymax = bounds
    return (xmin <= pt[0] <= xmax) and (ymin <= pt[1] <= ymax)


Let's visualize what the above does:

In [ ]:
seed=10
neps=1
rng = np.random.default_rng(seed) #random number generator
pts=rng.integers(low=0+neps,high=100-neps,size=(10,2),endpoint=True) 
b_vor=bounded_voronoi(pts,(0,100,0,100))

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
voronoi_plot_2d(b_vor,ax)
plt.show()

The above allows us to compute the centroids related to each original point, since they all have bounded Voronoi cells now.

Next, let's create a function that calculates the centroid for a polygon defined by an array of vertex. And we can implement the Lloyd's algorithm. 

#### The following code taken from the previous MIT version of the course, https://gvarnavides.com/generative-art-workshop-website/docs/01.25-Tuesday/space-filling-curves. Nice code!!! XD

In [ ]:
#Numba: Numba translates Python functions to optimized machine code at runtime. A simple way to speed up the code.
from numba import njit

# Converge to a centroidal voronoi diagram.
def centroidal_voronoi(pts, bounds, iters=1):
    N = len(pts) # the number of points to adjust.
    for k in range(iters):
        vor = bounded_voronoi(pts, bounds)
        
        # find the polygons corresponding to the original points' Voronoi cells. Each
        # polygon is represented by a list of vertex coordinates, with the last matching
        # the first to form a closed loop. To use njit, we will need to unravel the list
        # of lists of vertex ids and use a pointer to indicate the start of each new
        # polygon's vertex set.
        vert_ids = [vor.regions[r]+[vor.regions[r][0]] for r in vor.point_region[:N]]
        vert_ptr = np.cumsum(np.array([0]+[len(verts) for verts in vert_ids]))
        vert_ids = np.array([v for verts in vert_ids for v in verts])
        vert_pos = np.array(vor.vertices)
        
        # update the current set of points to the centroids of the cells.
        pts = get_centroids(vert_ids, vert_ptr, vert_pos)
    return pts

@njit
def centroid(pts):
    # compute the centroid of a closed polygon with coordinates given by pts.
    # the first and last coordinates should be the same.
    area=0; c=np.zeros(2)
    for i in range(len(pts)-1):
        s = pts[i,0]*pts[i+1,1]-pts[i+1,0]*pts[i,1]
        area += s
        c+=s*(pts[i]+pts[i+1])
    c = np.abs(c)/(3*np.abs(area))
    return c

@njit
def get_centroids(vert_ids, vert_ptr, vert_pos):
    N = len(vert_ptr)-1
    centroids = np.zeros((N,2))
    for i in range(N):
        verts = vert_ids[vert_ptr[i]:vert_ptr[i+1]]
        pos = vert_pos[verts]
        centroids[i] = centroid(pos)
    return centroids

Let's use the above code on some random points, and run some iterations of Lloyd's algorithm, and visualize the resulting Voronoi diagram. We can also try the resulting bounded CVT on our image!

In [ ]:
#Generate Np random points in the image img range.
points = [] 
eps=1
Np=5000
for i in range(Np):
    points.append([rng.integers(low=eps,high=dim_x-eps,endpoint=False), \
                   rng.integers(low=eps,high=dim_y-eps,endpoint=False)])
points = np.array(points)
#Define the bounds
bounds=(0,dim_x,0,dim_y)
#Get the set of final points after some iterations of Lloyd's algorithm
final_points=centroidal_voronoi(points, bounds, iters=200)
#Obtain the Voronoi diagram with the final set of points
b_vor_final=bounded_voronoi(final_points,bounds)

In [ ]:
#Visulize the bounded Voronoi diagram with the image
fig = plt.figure(figsize=(20,int(20*dim_y/dim_x)))
ax = fig.add_subplot(111)
ax.imshow(img)
voronoi_plot_2d(b_vor_final,ax=ax)
plt.show()

In [ ]:
#Visualize our intereted domain
fig = plt.figure(figsize=(20,int(20*dim_y/dim_x)))
ax = fig.add_subplot(111)
ax.imshow(img)
voronoi_plot_2d(b_vor_final,ax)
plt.xlim(0,dim_x)
plt.ylim(0,dim_y)
plt.show()

In [ ]:
# plot Voronoi diagram, and fill finite regions with color mapped from speed value

# set figure size
fig = plt.figure(figsize=(10,int(10*dim_y/dim_x)))
ax = fig.add_subplot(111)

# plot Voronoi diagram
voronoi_plot_2d(b_vor_final, ax=ax,show_points=False, show_vertices=False, line_width=0.1, line_alpha=0.5)

# for each input/generator point, fill color of its Voronoi cell
for pi in range(0,Np):
    
    # generator point coordinate [px,py]
    px=points[pi,0]
    py=points[pi,1]
    
    # if the point is inside the image range
    if px>=0 and px<dim_x and py>=0 and py<dim_y:
        
        #get the corresponding RGB value, convert to float tuple
        rgb_img= tuple(img[py,px]/255)
        
        #get the Voronoi region associated with the input/generator point
        ri=b_vor_final.point_region[pi]
        
        #if the Voronoi region is valid and finite, 
        #fill the polygon defined by the Voronoi cell with the color
        if ri!=-1:
            region = b_vor_final.regions[ri]
            if not -1 in region:
                polygon = [b_vor_final.vertices[i] for i in region]
                ax.fill(*zip(*polygon), color=rgb_img)

#hide plot axis
ax.axis('off')
#set plotting x range and y range
plt.xlim((0,dim_x))
plt.ylim((0,dim_y))
plt.show()

## 3.3. Meshing algorithm

Our goal is to get a nice triangular mesh on a circular shape, and then do the image coloring on it. This can be break down into a few steps: 
- First, trim the original set of random points, and discard points outside of the circle (sdf>0), keep only points inside/on the circle (sdf<=0).
- Then, perform bounded CVT on this trimmed set of points. In each iteration:
    - if point moved outside of shape (i.e. centroid outside of shape), project point back onto the shape boundary. 
- Once we get the final set of points after some iterations of the Lloyd's algorithm, we may get the Delaunay triangulation on the set of points.
- Lastly, do the coloring with bilinear interpolation in the triangles.

## 3.3.1. Trim points inside shape

Suppose we want our circle to be centered at (600,450) with radius 400, which should roughly covers the puppy's face.

In [ ]:
#sdf related: 
x0=600
y0=450
r=400
#Generate random points in the image img range.
points = [] 
eps=0
Np=5000
#h0: charastertistic element lengthscale
h0=dim_x/np.sqrt(Np)
#Generate points
for i in range(Np):
    
    #Trim points and only keep points with SDF <= 0.
    x=rng.integers(low=-eps,high=dim_x+eps,endpoint=False)
    y=rng.integers(low=-eps,high=dim_y+eps,endpoint=False)
    if sdf_circle(x,y,x0,y0,r)<=0:
        points.append([x,y])

points = np.array(points)
Np=len(points)

In [ ]:
#Visualize the initial set of points
plt.figure(figsize=(10,10))
plt.plot(points[:,0],points[:,1], "o")

## 3.3.2. Project outside points back to geometry boundary

For a point lying outside of the geometry domain, its negative gradient direction points to the smallest distance towards the boundary (i.e. perpendicular to the tangent of the closest point on the geometry boundary.)

![projection.PNG](https://raw.githubusercontent.com/yue-sun/generative-art/master/02_tuesday/figs/projection.PNG)

In [ ]:
num_eps=0.1
def projection(x,y):
    xk=x
    yk=y
    sdf_old=sdf_circle(x,y,x0,y0,r)
    if sdf_old>0:
        dgradx=(sdf_circle(x+num_eps,y,x0,y0,r)-sdf_old)/num_eps
        dgrady=(sdf_circle(x,y+num_eps,x0,y0,r)-sdf_old)/num_eps
        xk=x-sdf_old*dgradx
        yk=y-sdf_old*dgrady
    return [xk,yk]

## 3.3.3. Lloyd's iterations and projection of outside points

In [ ]:
#Define the bounds
beps=5
bounds=(x0-r-eps,x0+r+beps,y0-r-beps,y0+r+beps)

In [ ]:
#Do Niter iterations of Lloyd's algorithm
Niter=200
old_pt=np.copy(points)
movement_thres=h0

#Iterations of Lloyd's algorithm
for i in range(Niter):
    #new points position, i.e. centroids
    new_pt=centroidal_voronoi(old_pt, bounds, iters=1)
    
    #loop through the set of new points, and if any point is outside of the geometry,
    #project it back to the geometry boundary
    for pi in range(0,Np):
        px=new_pt[pi,0]
        py=new_pt[pi,1]
        
        #project point if it still lies outside of geometry
        if sdf_circle(px,py,x0,y0,r)>0:
            pt_proj=projection(px,py)
            new_pt[pi,0]=pt_proj[0]
            new_pt[pi,1]=pt_proj[1]
    old_pt=new_pt

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(old_pt[:,0],old_pt[:,1], "o")

In [ ]:
#Get the set of final points after some iterations of Lloyd's algorithm
final_points=old_pt
#Obtain the Voronoi diagram with the final set of points
b_vor_final=bounded_voronoi(final_points,bounds)

In [ ]:
#Visualize our intereted domain
fig = plt.figure(figsize=(20,int(20*dim_y/dim_x)))
ax = fig.add_subplot(111)
ax.imshow(img)
voronoi_plot_2d(b_vor_final,ax)
plt.xlim(bounds[0],bounds[1])
plt.ylim(bounds[2],bounds[3])
plt.show()

## 3.3.4. Delaunay triangulation and coloring

Lastly, as before, obtain the Delaunay triangulation and do the bilinear coloring.

In [ ]:
tria_img = Delaunay(final_points)

In [ ]:
#View the triangulation overlaying the image
fig = plt.figure(figsize=(20,int(20*dim_y/dim_x)))
ax = fig.add_subplot(111)
ax.imshow(img,origin="lower")
plt.triplot(final_points[:,0],final_points[:,1], tria_img.simplices,"-o")
plt.xlim(bounds[0],bounds[1])
plt.ylim(bounds[2],bounds[3])
plt.show()

In [ ]:
ver_rgb = np.zeros((Np,3))
for pi in range(0,Np):
    # generator point coordinate [px,py]
    px=final_points[pi,0]
    py=final_points[pi,1]
    
    # if the point is inside the image range
    if px>=0 and px<dim_x and py>=0 and py<dim_y:
        
        #get the corresponding RGB value, convert to float tuple
        ver_rgb[pi,:]= img[int(py),int(px)]/255

In [ ]:
#create our own color map for use in tripcolor(...) plotting later
z = np.arange(len(ver_rgb))
cmap = clr.LinearSegmentedColormap.from_list(
    "mymap", ver_rgb, N=len(ver_rgb)
)

In [ ]:
#create a triangulation object for use in tripcolor(...) plotting later
triang_obj = mtri.Triangulation(final_points[:,0],final_points[:,1], tria_img.simplices)

In [ ]:
# plot Delaunay triangulation, and fill triangles inside the image with bilinear color interpolation

# set figure size
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111)

# plot Delaunay triangulation
ax.triplot(final_points[:,0],final_points[:,1],tria_img.simplices,linewidth=0.1,alpha=0.5,color="black")

#bilinear interpolation coloring
ax.tripcolor(triang_obj, z, shading="gouraud", cmap=cmap)

#hide plot axis
ax.axis('off')
#set plotting x range and y range
plt.xlim(bounds[0],bounds[1])
plt.ylim(bounds[2],bounds[3])
plt.show()

So cute!

# 4. Free creating!

Some potential variables to consider:
- image selection
- random seed
- number of points	
- SDF shape representation 
- solid coloring or bilinear coloring
- Get creative! Color scaling and creative manipulations of RGB values
- Use CVD meshing or not

More advanced:

- What if more complicated shapes? Shapes combination? Union, Intersection, Difference
- Shape with concaveness? Dealing with concaveness: Keep only triangles with centroids inside shape
- Meshing with non-uniform density fields
- others...
- Recommended readings: 

    https://people.sc.fsu.edu/~jburkardt/classes/urop_2016/du_faber_gunzburger.pdf
    
    http://persson.berkeley.edu/distmesh/persson04mesh.pdf

# ~ Thank You ~ #

## Gallery 

![cat_voro_cvd.PNG](https://raw.githubusercontent.com/yue-sun/generative-art/master/02_tuesday/figs/cat_voro_cvd.png)

Guess where the following places are?

![blue_hill_voro.PNG](https://raw.githubusercontent.com/yue-sun/generative-art/master/02_tuesday/figs/blue_hill_voro.png)

![fuji_voro.PNG](https://raw.githubusercontent.com/yue-sun/generative-art/master/02_tuesday/figs/fuji_voro.png)

![seoul_tria.PNG](https://raw.githubusercontent.com/yue-sun/generative-art/master/02_tuesday/figs/seoul_tria.png)